In [6]:
import hail as hl

hl.init(
        log="/test_toolbox.log",
        tmp_dir="gs://gnomad-tmp-30day",
    )

Loading BokehJS ...

/opt/conda/miniconda3/lib/python3.11/site-packages/hailtop/aiocloud/aiogoogle/user_config.py:43: UserWarning:

Reading spark-defaults.conf to determine GCS requester pays configuration. This is deprecated. Please use `hailctl config set gcs_requester_pays/project` and `hailctl config set gcs_requester_pays/buckets`.

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


SPARKMONITOR_LISTENER: Started SparkListener for Jupyter Notebook
SPARKMONITOR_LISTENER: Port obtained from environment: 52867
SPARKMONITOR_LISTENER: Application Started: application_1727699094620_0001 ...Start Time: 1727699722656


Running on Apache Spark version 3.5.0
SparkUI available at http://qh1-m.c.broad-mpg-gnomad.internal:43005
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.132-678e1f52b999
LOGGING: writing to /test_toolbox.log


In [11]:
def get_variant_count(
    ht: hl.Table,
    afs: list[float] = [0.01, 0.001],
    singletons: bool = False,
    doubletons: bool = False,
) -> dict:
    """
    Count variants with frequency <1%, <0.1%, and singletons (AC == 1).

    .. note:: This function works for gnomAD exomes and genomes datasets, not yet for
              gnomAD joint dataset, since the HT schema is slightly different.

    :param ht: Input Table.
    :param afs: List of allele frequencies cutoffs.
    :param singletons: Include singletons.
    :param doubletons: Include doubletons.
    :return: Dictionary with counts.
    """
    counts = {}

    # Filter to PASS variants.
    ht = ht.filter(hl.len(ht.filters) == 0)
    if singletons:
        n_singletons = ht.aggregate(hl.agg.count_where(ht.freq[0].AC == 1))
        counts["number of singletons"] = n_singletons
    if doubletons:
        n_doubletons = ht.aggregate(hl.agg.count_where(ht.freq[0].AC == 2))
        counts["number of doubletons"] = n_doubletons

    for af in afs:
        n_variants = ht.aggregate(hl.agg.count_where(ht.freq[0].AF < af))
        counts[f"number of variants with AF < {af}"] = n_variants

    # Count variants with frequency <1%, <0.1%, and singletons (AC == 1).
    return counts

# Get variant count

## Get variant count by AF for a release

In [8]:
from gnomad.resources.grch38.gnomad import public_release as v4_public_release

In [9]:
ht = v4_public_release("exomes").ht()

In [12]:
print(get_variant_count(ht))

{'number of variants with AF < 0.01': 68398090, 'number of variants with AF < 0.001': 67709028}


In [13]:
print(get_variant_count(ht, singletons=True, doubletons=True))

{'number of singletons': 34047562, 'number of doubletons': 10161819, 'number of variants with AF < 0.01': 68398090, 'number of variants with AF < 0.001': 67709028}


## Get variant count by AF for a gene

In [15]:
# Filter to interval, e.g. for ASH1L.
gene_interval = "chr1:155335268-155563162"

# Filter the exome release Hail Table to the ASH1L gene interval.
ht = hl.filter_intervals(ht, [hl.parse_locus_interval(gene_interval, reference_genome="GRCh38")])

print(get_variant_count(ht, singletons=True, doubletons=True))

{'number of singletons': 5282, 'number of doubletons': 1616, 'number of variants with AF < 0.01': 10733, 'number of variants with AF < 0.001': 10656}


# Filter to variants by VEP annotations

## Filter to LOF variants

In [16]:
from gnomad.utils.vep import filter_vep_transcript_csqs
# Filter to variants in ASH1L that are LOFTEE high-confidence (with no flags) in the MANE select transcript.
ht = filter_vep_transcript_csqs(
    ht, 
    synonymous=False, 
    mane_select=True,
    genes=["ASH1L"],
    match_by_gene_symbol=True,
    additional_filtering_criteria=[lambda x: (x.lof == "HC") & hl.is_missing(x.lof_flags)],
)

INFO (gnomad.utils.vep 928): Filtering to canonical transcripts
INFO (gnomad.utils.vep 931): Filtering to MANE Select transcripts...
INFO (gnomad.utils.vep 934): Filtering to Ensembl transcripts...
INFO (gnomad.utils.vep 940): Filtering to genes of interest...
INFO (gnomad.utils.vep 948): Filtering to variants with additional criteria...


## Filter to pLOF variants that we used to compute constraint metrics
pLOF variants meets the following requirements:
* High-confidence LOFTEE variants (without any flags),
* Only variants in the MANE Select transcript,
* PASS variants that are SNVs with MAF ≤ 0.1%, and
* Exome median depth ≥ 30

**Note: this number should match the number of observed pLOF SNVs on the gene page of gnomAD Browser.**

In [17]:
from gnomad.resources.grch38.gnomad import coverage

coverage_ht = coverage("exomes").ht()

#Filter to PASS SNVs with AF <= 0.1% and median exome depth ≥ 30.
ht = ht.filter(
    (hl.len(ht.filters) == 0) 
    & (ht.allele_info.allele_type == "snv")
    & (ht.freq[0].AF <= 0.001)
    & (coverage_ht[ht.locus].median_approx >= 30)
)

print(f"Number of variants: {ht.count()}")
ht.select(
    freq=ht.freq[0],
    csq=ht.vep.transcript_consequences[0].consequence_terms,
    coverage=coverage_ht[ht.locus],
).show(-1)

Number of variants: 18


+----------------+------------+---------+----------+---------+
| locus          | alleles    | freq.AC |  freq.AF | freq.AN |
+----------------+------------+---------+----------+---------+
| locus<GRCh38>  | array<str> |   int32 |  float64 |   int32 |
+----------------+------------+---------+----------+---------+
| chr1:155337668 | ["G","A"]  |     459 | 3.14e-04 | 1461270 |
| chr1:155337704 | ["G","A"]  |       6 | 4.10e-06 | 1461714 |
| chr1:155337735 | ["G","C"]  |       1 | 6.84e-07 | 1461694 |
| chr1:155338087 | ["A","T"]  |       1 | 6.85e-07 | 1459648 |
| chr1:155338161 | ["G","A"]  |       1 | 6.84e-07 | 1461884 |
| chr1:155349380 | ["T","A"]  |       1 | 6.84e-07 | 1461768 |
| chr1:155354631 | ["C","T"]  |       1 | 6.88e-07 | 1453416 |
| chr1:155357583 | ["A","T"]  |       1 | 6.84e-07 | 1461680 |
| chr1:155370984 | ["C","T"]  |       3 | 2.06e-06 | 1455726 |
| chr1:155415924 | ["C","A"]  |       2 | 1.48e-06 | 1352032 |
| chr1:155478020 | ["G","C"]  |       1 | 6.84e-07 | 1461888 |
| chr1:155478203 | ["C","T"]  |       1 | 6.84e-07 | 1461874 |
| chr1:155478439 | ["C","T"]  |       1 | 6.84e-07 | 1461890 |
| chr1:155478528 | ["G","A"]  |       1 | 6.84e-07 | 1461876 |
| chr1:155479767 | ["G","A"]  |       1 | 6.84e-07 | 1461858 |
| chr1:155479862 | ["G","T"]  |       1 | 6.84e-07 | 1461160 |
| chr1:155521291 | ["C","A"]  |       1 | 6.84e-07 | 1461862 |
| chr1:155521474 | ["C","A"]  |       1 | 6.84e-07 | 1461724 |
+----------------+------------+---------+----------+---------+

+-----------------------+-----------------------------+---------------+
| freq.homozygote_count | csq                         | coverage.mean |
+-----------------------+-----------------------------+---------------+
|                 int64 | array<str>                  |       float64 |
+-----------------------+-----------------------------+---------------+
|                     0 | ["stop_gained"]             |      2.99e+01 |
|                     0 | ["stop_gained"]             |      3.02e+01 |
|                     0 | ["stop_gained"]             |      3.00e+01 |
|                     0 | ["splice_donor_variant"]    |      3.16e+01 |
|                     0 | ["stop_gained"]             |      3.19e+01 |
|                     0 | ["stop_gained"]             |      3.21e+01 |
|                     0 | ["splice_acceptor_variant"] |      3.04e+01 |
|                     0 | ["splice_donor_variant"]    |      3.13e+01 |
|                     0 | ["splice_acceptor_variant"] |      3.12e+01 |
|                     0 | ["splice_acceptor_variant"] |      2.73e+01 |
|                     0 | ["stop_gained"]             |      3.26e+01 |
|                     0 | ["stop_gained"]             |      3.27e+01 |
|                     0 | ["stop_gained"]             |      3.28e+01 |
|                     0 | ["stop_gained"]             |      3.28e+01 |
|                     0 | ["stop_gained"]             |      3.30e+01 |
|                     0 | ["stop_gained"]             |      3.24e+01 |
|                     0 | ["stop_gained"]             |      3.30e+01 |
|                     0 | ["stop_gained"]             |      3.29e+01 |
+-----------------------+-----------------------------+---------------+

+------------------------+-------------------+-----------------+
| coverage.median_approx | coverage.total_DP | coverage.over_1 |
+------------------------+-------------------+-----------------+
|                  int32 |             int64 |         float64 |
+------------------------+-------------------+-----------------+
|                     30 |          21887528 |        1.00e+00 |
|                     30 |          22042372 |        1.00e+00 |
|                     30 |          21956914 |        1.00e+00 |
|                     31 |          23111319 |        1.00e+00 |
|                     31 |          23308434 |        1.00e+00 |
|                     32 |          23444999 |        1.00e+00 |
|                     31 |          2220971

# Get 'freq' for specific genetic ancestry groups

In [85]:
ht = v4_public_release("exomes").ht()

# Filter to interval, e.g. for ASH1L.
gene_interval = "chr1:155335268-155563162"

# Filter the exome release Hail Table to the ASH1L gene interval.
ht = hl.filter_intervals(ht, [hl.parse_locus_interval(gene_interval, reference_genome="GRCh38")])

# Filter to variants with adj.AC > 0 
ht = ht.filter(ht.freq[0].AC>0)

In [86]:
# For this example, we filter to the ancestry that we included in the FAF calculation
from gnomad.resources.grch38.gnomad import POPS_TO_REMOVE_FOR_POPMAX

POPS_TO_REMOVE_FOR_POPMAX["v4"]

{'ami', 'asj', 'fin', 'oth', 'remaining'}

In [87]:
from gnomad.utils.filtering import filter_arrays_by_meta

# Remove unwanted stratifications
items_to_filter1 = ['sex','downsampling','subset']
freq_meta1, array_exprs1 = filter_arrays_by_meta(
        ht.freq_meta,
        {
            **{a: ht[a] for a in ['freq']},
            "freq_meta_sample_count": ht.index_globals().freq_meta_sample_count,
        },
        items_to_filter=items_to_filter1,
        keep=False,
        combine_operator="or",
    )

In [88]:
# keep the necessary stratifications for your analysis 
items_to_filter2 = {'gen_anc':['ami', 'asj', 'fin', 'oth', 'remaining'], 'group':['raw']}

freq_meta2, array_exprs2 = filter_arrays_by_meta(
        freq_meta1,
        array_exprs1,
        items_to_filter=items_to_filter2,
        keep=False,
        combine_operator="or",
    )

In [89]:
freq_meta2.collect()

[[{'group': 'adj'},
  {'gen_anc': 'afr', 'group': 'adj'},
  {'gen_anc': 'amr', 'group': 'adj'},
  {'gen_anc': 'eas', 'group': 'adj'},
  {'gen_anc': 'mid', 'group': 'adj'},
  {'gen_anc': 'nfe', 'group': 'adj'},
  {'gen_anc': 'sas', 'group': 'adj'}]]

In [90]:
ht = ht.annotate(**{a: array_exprs2[a] for a in ['freq']})
ht = ht.annotate_globals(
        freq_meta=freq_meta2,
        freq_meta_sample_count=array_exprs2["freq_meta_sample_count"],
    )

In [91]:
populations = ['all', 'afr', 'amr', 'eas', 'mid', 'nfe', 'sas']
ht.select(**{pop: ht.freq[i] for i, pop in enumerate(populations)}).show(5)

+----------------+------------+--------+----------+--------+
| locus          | alleles    | all.AC |   all.AF | all.AN |
+----------------+------------+--------+----------+--------+
| locus<GRCh38>  | array<str> |  int32 |  float64 |  int32 |
+----------------+------------+--------+----------+--------+
| chr1:155335497 | ["A","C"]  |      1 | 1.79e-03 |    560 |
| chr1:155335570 | ["T","C"]  |      3 | 5.26e-03 |    570 |
| chr1:155335571 | ["TA","T"] |      3 | 5.26e-03 |    570 |
| chr1:155335746 | ["G","C"]  |      1 | 1.77e-03 |    564 |
| chr1:155335855 | ["G","A"]  |      1 | 1.74e-03 |    576 |
+----------------+------------+--------+----------+--------+

+----------------------+--------+---------+--------+----------------------+
| all.homozygote_count | afr.AC |  afr.AF | afr.AN | afr.homozygote_count |
+----------------------+--------+---------+--------+----------------------+
|                int64 |  int32 | float64 |  int32 |                int64 |
+----------------------+--------+---------+--------+----------------------+
|                    0 |      0 |      NA |      0 |                    0 |
|                    0 |      0 |      NA |      0 |                    0 |
|                    0 |      0 |      NA |      0 |                    0 |
|                    0 |      0 |      NA |      0 |                    0 |
|                    0 |      0 |      NA |      0 |                    0 |
+----------------------+--------+---------+--------+----------------------+

+--------+---------+--------+----------------------+--------+----------+
| amr.AC |  amr.AF | amr.AN | amr.homozygote_count | eas.AC |   eas.AF |
+--------+---------+--------+----------------------+--------+----------+
|  int32 | float64 |  int32 |                int64 |  int32 |  float64 |
+--------+---------+--------+----------------------+--------+----------+
|      0 |      NA |      0 |                    0 |      0 | 0.00e+00 |
|      0 |      NA |      0 |                    0 |      1 | 7.25e-03 |
|      0 |      NA |      0 |                    0 |      1 | 7.25e-03 |
|      0 |      NA |      0 |                    0 |      0 | 0.00e+00 |
|      0 |      NA |      0 |                    0 |      1 | 7.25e-03 |
+--------+---------+--------+----------------------+--------+----------+

+--------+----------------------+--------+---------+--------+
| eas.AN | eas.homozygote_count | mid.AC |  mid.AF | mid.AN |
+--------+----------------------+--------+---------+--------+
|  int32 |                int64 |  int32 | float64 |  int32 |
+--------+----------------------+--------+---------+--------+
|    138 |                    0 |      0 |      NA |      0 |
|    138 |                    0 |      0 |      NA |      0 |
|    138 |                    0 |      0 |      NA |      0 |
|    132 |                    0 |      0 |      NA |      0 |
|    138 |                    0 |      0 |      NA |      0 |
+--------+----------------------+--------+---------+--------+

+----------------------+--------+----------+--------+----------------------+
| mid.homozygote_count | nfe.AC |   nfe.AF | nfe.AN | nfe.homozygote_count |
+----------------------+--------+----------+--------+----------------------+
|                int64 |  int32 |  float64 |  int32 |                int64 |
+----------------------+--------+----------+--------+----------------------+
|                    0 |      0 | 0.00e+00 |      2 |                    0 |
|                    0 |      0 | 0.00e+00 |      2 |                    0 |
|                    0 |      0 | 0.00e+00 |      2 |                    0 |
|                    0 |      0 | 0.00e+00 |      2 |                    0 |
|                    0 |      0 | 0.00e+00 |      6 |                    0 |
+----------------------+--------+----------+--------+----------------------+

+--------+---------+--------+----------------------+
| sas.AC |  sas.AF | sas.AN | sas.homozygote_count |
+--------+---------+--------+---------------------